# Data Science Repository Trends on GitHub: An Insightful EDA

### Project Break - EDA
Autor: Lucas Perez Barone


Este notebook está diseñado para recopilar datos de repositorios a través de la API Classic de GitHub. 

La información general sobre la API se puede encontrar en el siguiente enlace: *[GitHub REST API Overview](https://docs.github.com/en/rest?apiVersion=2022-11-28)*

Comenzando con la importación de funciones creadas por el usuario y adecuadas a mis objetivos. El código tiene flexibilidad y puede ajustarse según sus necesidades.

In [2]:
from user_functions import *

Creación de **HEADERS** con autenticación mediante token individual y definición del endpoint HTTP a utilizar. 

La API de GitHub opera en varios repositorios distintos, por lo que si desea obtener datos de otra dirección, consulte la documentación para Endpoints: [GitHub REST API Endpoints](https://docs.github.com/en/rest/using-the-rest-api/getting-started-with-the-rest-api?apiVersion=2022-11-28#3-choose-an-endpoint-for-your-request).

In [3]:
from my_GH_token import my_GH_token

HEADERS = {'Authorization': f'token {my_GH_token}'}

base_url = 'https://api.github.com/'
endpoint = 'search/repositories'

## Filtros de Búsqueda
### Estos son los principales filtros que se pueden usar al buscar repositorios:

| **Parámetro**    | **Descripción**                                                                 | **Valores/Ejemplos**                      |
|------------------|---------------------------------------------------------------------------------|--------------------------------------------|
| **topic:**       | Filtra repositorios por tema (data-science, machine-learning, etc.)             | `data-science`, `machine-learning`, etc.   |
| **stars:>=**     | Filtra repositorios con al menos un número determinado de estrellas.            | `>=100`, `>=50`, etc.                      |
| **fork:**        | Filtra repositorios que son bifurcaciones de otros repositorios.                | `true`, `false`                            |
| **language:**    | Filtra repositorios por lenguaje de programación.                               | `python`, `javascript`, `ruby`, etc.       |
| **created:**    | Filtra repositorios creados en un rango de fechas específicas (inicio..fim)     | `year(4d)-month(2d)-day(2d)`/ `2022-05-15`, etc.|
| **updated:**    | Filtra repositorios actualizados hasta una fecha específica.                   | `year(4d)-month(2d)-day(2d)`/ `2022-05-01`, etc. |
| **is:**          | Filtra repositorios públicos o privados                                         | `public`, `private`                              |
| **sort:**        | Ordena los resultados por un campo específico.                                  | `stars`, `updated`, `forks`, `help-wanted-issues` |
| **order:**       | Define el orden de clasificación.                                               | `asc`, `desc`                                |

Estos son los principales parámetros que se pueden utilizar en una solicitud GET a través de la API para Repositorios. 

Si desea explorar nuevos filtros o diferentes solicitudes HTTP, visite el sitio web: [GitHub REST API - GET Repositories](https://docs.github.com/en/rest/repos/repos?apiVersion=2022-11-28#get-a-repository).

Definición del rango de años a ser recopilados a través de la API. Este es un argumento de la función del usuario get_repositories_by_year.

In [ ]:
date_range = range(2010, 2025)  # Rango de año a adquirir

La obtención de datos a través de la API para repositorios devuelve 81 variables/columnas de datos, siendo estas:

[`id`, `node_id`, `name`, `full_name`, `private`, `owner`, `html_url`, `description`, `fork`, `url`, `forks_url`, `keys_url`, `collaborators_url`, `teams_url`, `hooks_url`, `issue_events_url`, `events_url`, `assignees_url`, `branches_url`, `tags_url`, `blobs_url`, `git_tags_url`, `git_refs_url`, `trees_url`, `statuses_url`, `languages_url`, `stargazers_url`, `contributors_url`, `subscribers_url`, `subscription_url`, `commits_url`, `git_commits_url`, `comments_url`, `issue_comment_url`, `contents_url`, `compare_url`, `merges_url`, `archive_url`, `downloads_url`, `issues_url`, `pulls_url`, `milestones_url`, `notifications_url`, `labels_url`, `releases_url`, `deployments_url`, `created_at`, `updated_at`, `pushed_at`, `git_url`, `ssh_url`, `clone_url`, `svn_url`, `homepage`, `size`, `stargazers_count`, `watchers_count`, `language`, `has_issues`, `has_projects`, `has_downloads`, `has_wiki`, `has_pages`, `has_discussions`, `forks_count`, `mirror_url`, `archived`, `disabled`, `open_issues_count`, `license`, `allow_forking`, `is_template`, `web_commit_signoff_required`, `topics`, `visibility`, `forks`, `open_issues`, `watchers`, `default_branch`, `permissions`, `score`]

No entraré en el detalle de discutir cada una de ellas. Para más información, consulta la documentación oficial: [GitHub REST API Repositories - GET Content](https://docs.github.com/en/rest/repos/contents?apiVersion=2022-11-28).

Tras el estudio de las variables, se decidió recopilar solo algunas columnas, reduciendo así el peso y la complejidad del conjunto de datos, descartando información innecesaria para mis análisis.

In [4]:
selected_columns = [
    'id', 'name', 'full_name', 'owner', 'stargazers_count',
    'forks_count', 'watchers_count', 'open_issues_count', 'topics',
    'created_at', 'language', 'has_issues', 'has_projects','has_downloads', 'has_discussions', 
    'has_wiki', 'license', 'score'
]

| **Columna**            | **Descripción**                                                                                                  | **dtype** |
|------------------------|------------------------------------------------------------------------------------------------------------------|-----------|
| `id`                   | El identificador único del repositorio.                                                                          |     int64      |
| `name`                 | El nombre del repositorio.                                                                                       |      object    |
| `full_name`            | El nombre completo del repositorio, incluyendo el nombre del propietario.                                        |     object     |
| `owner`                | Un objeto que contiene información sobre el propietario del repositorio                                          |     object     |
| `stargazers_count`     | El número de estrellas (stars) que el repositorio ha recibido.                                                   |     int64      |
| `forks_count`          | El número de forks (copias) realizadas a partir del repositorio.                                                 |     int64      |
| `watchers_count`       | El número de observadores que están siguiendo las actividades del repositorio.                                   |     int64      |
| `open_issues_count`    | El número de problemas (issues) abiertos en el repositorio.                                                      |    int64       |
| `topics`               | Una lista de temas y palabras clave asociados al repositorio.                                                    |      object    |
| `created_at`           | La fecha y hora en que el repositorio fue creado.                                                                | datetime64[ns, UTC]|
| `language`             | El principal lenguaje de programación utilizado en el repositorio.                                               |      object   |
| `has_issues`           | Valor que indica si el repositorio tiene issues (problemas) habilitados.                                         |        bool   |
| `has_projects`         | Valor que indica si el repositorio tiene proyectos habilitados.                                                  |      bool     |
| `has_downloads`        | Valor que indica si el repositorio permite la descarga de archivos.                                              |       bool    |
| `has_discussions`      | Valor que indica si el repositorio tiene discusiones habilitadas.                                                |      bool     |
| `has_wiki`             | Valor que indica si el repositorio tiene una wiki habilitada.                                                    |      bool     |
| `license`              | El tipo de licencia bajo la cual el repositorio está disponible.                                                 |      object   |
| `score`                | Un valor numérico (0-1) que representa la relevancia de un repositorio en los resultados de una búsqueda.         |     int64    |

La función a continuación recibe tres argumentos:

HEADERS, que incluye el token de autenticación obligatorio y cualquier otro filtro que desees, como el tipo de aplicación;

Rango de fechas (date_range), que en este caso está diseñado para ser un rango de años, recopilando todos los datos dentro de ese período. Si necesitas otro tipo de rango de fechas, como meses o días, ajústalo directamente en el archivo user_functions.py;

Columnas seleccionadas (selected_columns), siendo este argumento una lista con todas las columnas deseadas para ser extraídas y guardadas. Si deseas extraer todas las columnas, simplemente no pases este argumento.

Si todos los argumentos son pasados correctamente, la función iniciará una serie de bucles automatizados para recopilar la información según los filtros y variables seleccionadas, dentro de todo el rango de fechas, y devolverá un archivo .json guardado en la misma carpeta en la que se está ejecutando. El nombre de los archivos será repositories_{year}, generando así un archivo .json para cada año procesado.

La función está programada para hacer pausas entre las solicitudes con el fin de no sobrecargar la API y evitar superar el límite de solicitudes (Rate Limit)

In [ ]:
get_repositories_by_year(HEADERS, date_range, selected_columns)

La celda de abajo está automatizada para recoger todos los archivos dentro del rango de fechas estipulado, concatenarlos y guardarlos en un único archivo `.json`.

In [38]:
date_range = range(2010, 2025)  # Rango de año
all_repositories = []

for year in date_range:
    filename = f'./data/repositories_{year}.json'
    
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            data = json.load(file)
            all_repositories.extend(data)  # Agrega los repositorios del año al total
            print(f"Repositorios de {year} cargados con éxito.")
    except FileNotFoundError:
        print(f"Archivo {filename} no encontrado. Saltando...")

# Guarda todos los repositorios concatenados en un nuevo archivo JSON
with open('repositories_2010_to_2014.json', 'w', encoding='utf-8') as output_file:
    json.dump(all_repositories, output_file, ensure_ascii=False, indent=4)

print("Repositorios de 2010 a 2014 concatenados y guardados en 'repositories_2010_to_2014.json'.")

Repositorios de 2010 cargados con éxito.
Repositorios de 2011 cargados con éxito.
Repositorios de 2012 cargados con éxito.
Repositorios de 2013 cargados con éxito.
Repositorios de 2014 cargados con éxito.
Repositorios de 2015 cargados con éxito.
Repositorios de 2016 cargados con éxito.
Repositorios de 2017 cargados con éxito.
Repositorios de 2018 cargados con éxito.
Repositorios de 2019 cargados con éxito.
Repositorios de 2020 cargados con éxito.
Repositorios de 2021 cargados con éxito.
Repositorios de 2022 cargados con éxito.
Repositorios de 2023 cargados con éxito.
Repositorios de 2024 cargados con éxito.
Repositorios de 2010 a 2014 concatenados y guardados en 'repositories_2010_to_2014.json'.
